In [112]:
import pandas as pd
from bs4 import BeautifulSoup
import requests
import numpy as np
import math
from datetime import datetime 
import hashlib
import time
from email.message import EmailMessage
import sys
import os
path = os.getcwd()
pathing = sys.path.append(path)
sys.path.append(os.path.abspath('.')) 

from internal_modules.email import send_email

In [113]:
aces_individual = 'https://www.ncaa.com/stats/volleyball-men/d1/current/individual/532'

In [114]:
service = requests.get(aces_individual)

In [115]:
collecting = service.content
information_aces = BeautifulSoup(collecting, 'html.parser')
information_aces.prettify()

date_updated = '<em class="placeholder">Monday, January 27, 2025 8:08 am</em>'

In [116]:
# developing a function to help us determine when we can check the last update

def last_update():
    url_aces = 'https://www.ncaa.com/stats/volleyball-men/d1/current/individual/532'
    service = requests.get(url_aces)
    collecting = service.content
    information_aces = BeautifulSoup(collecting, 'html.parser')
    most_recent_date = information_aces.find_all('em')[1].text.strip()
    if len(most_recent_date) <= 1:
        return f'There has been an issue with the retrieving of the data. Contact the Author to resolve the issue.'
    else:
        time_stamp = datetime.strptime(most_recent_date, "%A, %B %d, %Y %I:%M %p") 
    try: 
        return f'The last update for the NCAA page was {time_stamp}'
    except Exception as error:
        return f'There was an error at {error}, solve it to be able to parse the last updated date.'
    return time_stamp

last_update = last_update()
time_stamp

def checking_update():
    while True:
        time.sleep(24 * 60 * 80)  # Wait for 60 seconds between checks
        
        # Fetch the specific part
        last_update = last_update()
        # Compare with the saved content
        if last_update != time_stamp:
            print("The monitored part of the website has been updated!")
            saved_content = current_content  
        else:
            print("There has not been an update yet.")

In [117]:
last_updated = information_aces.find_all('em')[0]
# this is the going to be the date of when the page is updated
most_recent_date = information_aces.find_all('em')[1].text.strip()
most_recent_date
time_stamp = datetime.strptime(most_recent_date, "%A, %B %d, %Y %I:%M %p") 
print(time_stamp)


2025-01-28 08:58:00


In [118]:
# in order to make the data frame we want to make, we will need to find all the headers and columns we were using earlier
columns_aces = information_aces.find('tr').text
columns_aces = columns_aces.replace('\n',' ').strip().split()
df = pd.DataFrame(columns = columns_aces)
df['Per Set'] = df['Per'].astype(str) + df['Set'].astype(str)
df = df.drop(['Per','Set'], axis = 1)

In [119]:
# putting everything all together

table_data = information_aces.find_all('tr')

valid_ranking = 0

for item in table_data[1:]:
    vb = item.find_all('td')
    pieced = [info.text.strip() for info in vb]
    if pieced[0] == '-':
        valid_ranking += 1
        pieced[0] = str(valid_ranking)
    else:
        valid_ranking= int(pieced[0])
    length = len(df)
    df.loc[length] = pieced


# changing the data types
df['S'] = df['S'].astype(int)
df['Aces'] = df['Aces'].astype(int)
df['Per Set'] = df['Per Set'].astype(float)
df['Rank'] = df['Rank'].astype(int)

In [120]:
# we usually will use .3 to .5 for the best serving teams
# will research more on the 

average = df[(df['Per Set'] > .3) & (df['Per Set'] < .5) & (df['Aces'] >= 10) & (df['S'] > int(20))]
average

df

,Rank,Name,Team,Cl,Height,S,Aces,Per Set
0,1,Moni Nikolov,Long Beach St.,Fr.,6-8,21,22,1.048
1,2,Hilir Henno,UC Irvine,Sr.,6-8,17,17,1.000
2,3,Aleksa Mandic,Sacred Heart,So.,6-5,8,8,1.000
3,4,Kristian Titriyski,Hawaii,Fr.,6-5,18,15,0.833
4,5,Parker Van Buren,Loyola Chicago,Sr.,6-9,21,17,0.810
5,6,Bryce Dvorak,UC San Diego,Sr.,6-4,26,21,0.808
6,7,Maxim Grigoriev,UC Irvine,Jr.,6-4,17,13,0.765
7,8,Ty Carson,Menlo,Fr.,6-4,19,14,0.737
8,9,Donovan Constable,CSUN,Sr.,6-2,25,17,0.680
9,10,Cole Hartke,Pepperdine,Fr.,-,11,7,0.636


In [121]:
# current status of project: 
# I am looking to do some research on the historcally best teams with their serving percentages and the overall 
# performance of the teams

# Automation should come next given the finalized of formulaic approach I decide to take